In [1]:
import pandas as pd 
import scipy.stats as st
import numpy as np


# Mostrar todo el contenido de las celdas (sin recortes)
pd.set_option('display.max_colwidth', None)

# Mostrar todas las columnas 
pd.set_option('display.max_columns', None)

In [2]:
ruta = "/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/clasificacion_mia2025.csv"

dp = pd.read_csv(ruta)

In [3]:
dp = dp.drop(columns=["Unnamed: 0"])

dp["store_name"] = dp["store_name"].replace({"PRINCIPAL COTA":"MOSTRADOR COTA"})

In [4]:
stock_seguridad = {
    "SUCURSAL CALLE 6": 50.00,
    "SUCURSAL VALLADOLID": 50.00,
    "SUCURSAL NORTE": 50.00,
    "SUCURSAL MEDELLIN": 150.00,
    "MOSTRADOR COTA": 300.00,
    "SUCURSAL BARRANQUILLA": 150.00,
    "SUCURSAL BUCARAMANGA": 150.00,
    "SUCURSAL CALI": 150.00,
   # "ANTIOQUEÑA DE LUBRICANTES SGP SAS": 300.00
}

reorden_ideal = {
    "SUCURSAL CALLE 6": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL VALLADOLID": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL NORTE": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL MEDELLIN": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    #"ANTIOQUEÑA DE LUBRICANTES SGP SAS": {"Punto de reorden": 5.00, "Inventario ideal": 7.00},
    "SUCURSAL BARRANQUILLA": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "SUCURSAL BUCARAMANGA": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "MOSTRADOR COTA": {"Punto de reorden": 2.50, "Inventario ideal": 3.00},
    "SUCURSAL CALI": {"Punto de reorden": 7.00, "Inventario ideal": 9.00}, # Tiene para 7 semnaas de inventario , y cubre el inventario 9 semanas
}


In [5]:
dp.head(10)

,store_name,product_ref,Año,semana,Ema,Costo unitario,Venta costo EMA,Venta costo %,Acumulado costo,Desviacion EMA,Desviacion EMA %,acumulado_cantidad,coef_ventas,acumulado_desviacion,acumulado_desviacion_p,acumulado_combinado,Clasificacion,Nivel de servicio
0,MOSTRADOR COTA,DCS10536137,2025.0,31,27.875727,51663.09,1440146.19,0.0086,0.0086,6.624672e+05,0.46,0.01,3.934426,0.46,1.0,0.23,AAA,0.999
1,MOSTRADOR COTA,BCE00606125,2025.0,32,26.149581,20564.31,537748.09,0.0032,0.0844,8.262583e+11,1536515.46,0.01,0.000000,1536515.92,0.0,0.03,AAA,0.999
2,MOSTRADOR COTA,DCS10536137,2025.0,32,25.088154,51663.09,1296131.56,0.0077,0.0163,4.277234e+05,0.33,0.02,2.702703,1536516.25,0.8,0.20,AAA,0.999
3,MOSTRADOR COTA,DCS10536137,2025.0,28,24.298666,51663.09,1255344.17,0.0075,0.0238,0.000000e+00,0.00,0.02,0.000000,1536516.25,0.0,0.02,AAA,0.999
4,MOSTRADOR COTA,DCS10536137,2025.0,42,22.763445,51663.09,1176029.91,0.0070,0.0308,7.173782e+05,0.61,0.02,5.732484,1536516.86,1.0,0.24,AAA,0.999
5,MOSTRADOR COTA,DCS10536137,2025.0,33,22.579339,51663.09,1166518.42,0.0070,0.0378,4.316118e+05,0.37,0.03,3.243243,1536517.23,1.0,0.25,AAA,0.999
6,MOSTRADOR COTA,DCS10536137,2025.0,29,22.068799,51663.09,1140142.35,0.0068,0.0446,4.332541e+05,0.38,0.03,2.000000,1536517.61,0.8,0.21,AAA,0.999
7,MOSTRADOR COTA,BCE00606125,2025.0,33,20.919665,20564.31,430198.48,0.0026,0.1010,1.935893e+05,0.45,0.04,6.000000,1536518.06,1.0,0.27,AAA,0.999
8,MOSTRADOR COTA,DCS10536137,2025.0,30,19.861919,51663.09,1026128.11,0.0061,0.0507,4.309738e+05,0.42,0.04,2.950820,1536518.48,0.8,0.22,AAA,0.999
9,MOSTRADOR COTA,DCS10536137,2025.0,34,18.063471,51663.09,933214.73,0.0056,0.0563,3.732859e+05,0.40,0.04,3.783784,1536518.88,1.0,0.26,AAA,0.999


## INVENTARIO DE SEGURIDAD

In [6]:
# Copia del df original
df_std = dp.copy()

# Ordenar datos por tienda, producto, año y semana
df_std = df_std.sort_values(by=["store_name", "product_ref", "Año", "semana"])

# Calcular la desviación móvil (rolling) de EMA con ventana de 12 semanas
df_std["Desviacion_EMA_calc"] = (
    df_std.groupby(["store_name", "product_ref"])["Ema"]
    .transform(lambda x: x.rolling(window=12, min_periods=1).std())
)

# Calcular también la media móvil del EMA (opcional, para validación)
df_std["Ema_media"] = (
    df_std.groupby(["store_name", "product_ref"])["Ema"]
    .transform(lambda x: x.rolling(window=12, min_periods=1).mean())
)

# === INVENTARIO IDEAL y PUNTO DE REORDEN ===
# Diccionario con los dos parámetros (en semanas de cobertura)
reorden_ideal = {
    "SUCURSAL CALLE 6": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL VALLADOLID": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL NORTE": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL MEDELLIN": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "SUCURSAL BARRANQUILLA": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "SUCURSAL BUCARAMANGA": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "MOSTRADOR COTA": {"Punto de reorden": 2.50, "Inventario ideal": 3.00},
    "SUCURSAL CALI": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
}

# Extraer columnas de reorden e ideal
df_std["lt_reorden"] = df_std["store_name"].map(lambda x: reorden_ideal.get(x, {}).get("Punto de reorden", np.nan))
df_std["lt_ideal"] = df_std["store_name"].map(lambda x: reorden_ideal.get(x, {}).get("Inventario ideal", np.nan))


# Calcular Z-score según nivel de servicio
df_std["Z"] = df_std["Nivel de servicio"].apply(lambda x: st.norm.ppf(x))

# === STOCK DE SEGURIDAD (rolling 12 semanas) ===
df_std["inventario_seguridad"] = (
    df_std["Z"] * df_std["Desviacion_EMA_calc"] * np.sqrt(df_std["lt_ideal"])
)

# Inventario ideal (nivel máximo recomendado)
df_std["inventario_ideal"] = (df_std["Ema"] * df_std["lt_ideal"]) + df_std["inventario_seguridad"]


# Punto de reorden (cuándo pedir)
df_std["punto_reorden"] = (df_std["Ema"] * df_std["lt_reorden"]) + df_std["inventario_seguridad"]



# === Redondear columnas numéricas ===
df_std["inventario_seguridad"] = df_std["inventario_seguridad"].round(2)
df_std["inventario_ideal"] = df_std["inventario_ideal"].round(2)
df_std["punto_reorden"] = df_std["punto_reorden"].round(2)




df_std = df_std.drop(columns=["Desviacion_EMA_calc","Ema_media","lt_reorden","lt_ideal","Z"])

# TARDA 2 MINUTOS 

In [7]:
df_std[(df_std["semana"]==41) & (df_std["store_name"]=="SUCURSAL BARRANQUILLA")].sort_values(by=["Ema"],ascending=False).head(10)

,store_name,product_ref,Año,semana,Ema,Costo unitario,Venta costo EMA,Venta costo %,Acumulado costo,Desviacion EMA,Desviacion EMA %,acumulado_cantidad,coef_ventas,acumulado_desviacion,acumulado_desviacion_p,acumulado_combinado,Clasificacion,Nivel de servicio,inventario_seguridad,inventario_ideal,punto_reorden
53921,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,41,54.888271,13771.02,755867.48,0.0008,0.1096,173849.5,0.23,0.00,4.512195,0.41,1.0,0.26,AAA,0.999,92.03,586.02,476.25
53924,SUCURSAL BARRANQUILLA,DAB14570025,2025.0,41,51.032786,9917.17,506100.81,0.0006,0.1945,146769.2,0.29,0.01,4.633803,1.23,1.0,0.30,AAA,0.999,89.59,548.89,446.82
53960,SUCURSAL BARRANQUILLA,BCS00035125,2025.0,41,32.724550,20581.34,673515.09,0.0007,0.1252,94292.1,0.14,0.06,3.037406,8.42,1.0,0.31,AAA,0.999,36.70,331.22,265.77
53969,SUCURSAL BARRANQUILLA,BCS00025125,2025.0,41,30.081214,31057.73,934254.22,0.0010,0.0966,93425.4,0.10,0.07,2.296530,9.62,0.8,0.26,AAA,0.999,30.78,301.51,241.35
53973,SUCURSAL BARRANQUILLA,BLS00037125,2025.0,41,29.398848,32494.89,955312.33,0.0010,0.0956,95531.2,0.10,0.07,1.668687,10.02,0.8,0.26,AAA,0.999,33.60,298.19,239.39
53980,SUCURSAL BARRANQUILLA,DAB02772025,2025.0,41,28.498017,10883.85,310168.14,0.0003,0.3203,62033.6,0.20,0.08,3.922879,10.91,1.0,0.39,AAA,0.999,51.93,308.41,251.41
53995,SUCURSAL BARRANQUILLA,BCS00060125,2025.0,41,25.190084,30382.98,765349.82,0.0008,0.1072,130109.5,0.17,0.09,2.935484,2052811.85,0.8,0.28,AAA,0.999,32.96,259.67,209.29
53996,SUCURSAL BARRANQUILLA,BCS00249125,2025.0,41,25.108218,27389.88,687711.08,0.0008,0.1216,116910.9,0.17,0.10,3.287879,2052812.02,1.0,0.33,AAA,0.999,34.37,260.34,210.13
53999,SUCURSAL BARRANQUILLA,BCS10035125,2025.0,41,24.970648,20902.34,521944.97,0.0006,0.1837,93950.1,0.18,0.10,3.195021,2052812.20,1.0,0.34,AAA,0.999,36.50,261.23,211.29
54010,SUCURSAL BARRANQUILLA,DAB02666025,2025.0,41,23.548155,11371.65,267781.38,0.0003,0.3554,42845.0,0.16,0.11,2.317241,3387830.67,0.8,0.37,AAA,0.999,25.41,237.35,190.25


# INVENTARIO ACTUAL

In [8]:
ruta = "/home/donsson/proyectos/INVENTARIO/inventario_limpio.xlsx"

inv = pd.read_excel(ruta)

inv["product_ref"] = inv["Complete name"].str.extract(r"\[([A-Z0-9]+)\]")

In [9]:
inv.head(10)

,Sucursal,Estado,Codigo_interno,Complete name,Clase,Marca,categoria_producto,Cantidad,Costo unitario,Costo total,product_ref
0,PRINCIPAL COTA,Existencias,004 DA4207,[DAE04207004] DA4207 FILTRO AIRE SEGURIDAD KOMATSU (004 AF574SY),C,AUT*PARTS,Filtro,1.0,1.00,1.00,DAE04207004
1,PRINCIPAL COTA,Existencias,004 G314,[DLE00314004] G314 FILTRO ACEITE ESPECIAL (004 F62H),C,AUT*PARTS,Filtro,2.0,1890.00,3780.00,DLE00314004
2,PRINCIPAL COTA,Existencias,004 G401,[DLE00401004] G401 FILTRO ACEITE FIAT-ALLIS 75215486 (004 F75215486),C,AUT*PARTS,Filtro,7.0,1.00,7.00,DLE00401004
3,SUCURSAL MEDELLIN,Existencias,004 GA128,[DAE05128004] GA128 FILTRO AIRE FIAT DATSUN (004 GA128),C,AUT*PARTS,Filtro,2.0,1694.96,3389.92,DAE05128004
4,PRINCIPAL COTA,Existencias,004 GA128,[DAE05128004] GA128 FILTRO AIRE FIAT DATSUN (004 GA128),C,AUT*PARTS,Filtro,1.0,1694.96,1694.96,DAE05128004
5,PRINCIPAL COTA,Existencias,004 GA158,[DAE05158004] GA158 FILTRO AIRE RENULT 4 Y 6. (004 GA158),C,AUT*PARTS,Filtro,1.0,1722.50,1722.50,DAE05158004
6,PRINCIPAL COTA,Existencias,004 GA182,[DAE05182004] GA182 FILTRO AIRE SIMCA 1000 Y 1300 (004 GA182),C,AUT*PARTS,Filtro,1.0,1462.50,1462.50,DAE05182004
7,PRINCIPAL COTA,Existencias,004 GA321,[DAE05321004] GA321 FILTRO AIRE MONZA (004 GA321),C,AUT*PARTS,Filtro,2.0,1716.00,3432.00,DAE05321004
8,SUCURSAL BUCARAGAMNGA,Existencias,004 GS050,"[DLS00050004] GS050 FILTRO ACEITE SWIFT,SPRINT. (004 GS050)",C,AUT*PARTS,Filtro,1.0,6293.12,6293.12,DLS00050004
9,SUCURSAL NORTE,Existencias,004 GS050,"[DLS00050004] GS050 FILTRO ACEITE SWIFT,SPRINT. (004 GS050)",C,AUT*PARTS,Filtro,1.0,6293.12,6293.12,DLS00050004


In [10]:
inv["Cantidad"].sum() 

np.float64(311222.656)

# EVALUACION Y COMPARACION

In [11]:
import xmlrpc.client
import pandas as pd


# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


#COSTO DE LOS PRODUCTOS

productos_ema = models.execute_kw(
    db, uid, password,
    "cs.analisis.costo", "search_read",[],
    #[
    #    [["semana","=",36]]
    #],
    {"fields": ["sucursal_id","product_name","ano","semana","ema","producto_costo_unitario","punto_reorden","clasificacion","almacen_id","inventario_seguridad","inventario_ideal" ]}
)

df = pd.DataFrame(productos_ema)

def extract_id(val):
    if isinstance(val, (list, tuple)) and len(val) > 0:
        return val[0]
    return None

def extract_name(val):
    if isinstance(val, (list, tuple)) and len(val) > 1:
        return val[1]
    return str(val)

df.rename(columns={"product_name": "product_ref"}, inplace=True)


df["store_id_num"] = df["sucursal_id"].apply(extract_id)
df["store_name"]   = df["sucursal_id"].apply(extract_name)

df["almacen_id_num"] = df["almacen_id"].apply(extract_id)
df["almacen_name"]   = df["almacen_id"].apply(extract_name)


In [12]:
df_41 = df_std[df_std["semana"]==41]

df_41["Clasificacion_mia"] = df_41["Clasificacion"]

/tmp/ipykernel_21587/2500382178.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_41["Clasificacion_mia"] = df_41["Clasificacion"]


In [13]:
# --- MERGE LIMPIO PARA COMPARAR ---
df_merge_simple = pd.merge(
    df_41[['store_name', 'product_ref', 'Clasificacion_mia', 
            'inventario_seguridad', 'inventario_ideal', 'punto_reorden']],
    df[['store_name', 'product_ref', 'clasificacion', 
         'inventario_seguridad', 'inventario_ideal', 'punto_reorden']],
    on=['store_name', 'product_ref'],
    how='inner',
    suffixes=('_std', '_real')
)

# --- COMPARACIÓN DE CLASIFICACIÓN ---
df_merge_simple['match_clasificacion'] = (
    df_merge_simple['Clasificacion_mia'] == df_merge_simple['clasificacion']
)

# --- OPCIONAL: ORDENAR PARA REVISAR FÁCIL ---
df_merge_simple = df_merge_simple.sort_values(by=['store_name', 'product_ref'])

# --- MOSTRAR SOLO LAS COLUMNAS RELEVANTES ---
cols = [
    'store_name', 'product_ref',
    'Clasificacion_mia', 'clasificacion', 'match_clasificacion',
    'inventario_seguridad_std', 'inventario_seguridad_real',
    'inventario_ideal_std', 'inventario_ideal_real',
    'punto_reorden_std', 'punto_reorden_real'
]

df_merge_simple = df_merge_simple[cols]



df_merge_simple.to_excel("/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/diferencias_ema.xlsx")

In [14]:
df_merge_simple.sample(10)

,store_name,product_ref,Clasificacion_mia,clasificacion,match_clasificacion,inventario_seguridad_std,inventario_seguridad_real,inventario_ideal_std,inventario_ideal_real,punto_reorden_std,punto_reorden_real
16039,SUCURSAL MEDELLIN,BLE00767125,C,C,True,0.0,0.000000,0.00,0,0.00,0
14970,SUCURSAL MEDELLIN,BCS00428125,C,C,True,0.0,0.000000,0.00,0,0.00,0
26889,SUCURSAL VALLADOLID,DLS10286189,C,C,True,0.0,0.000000,0.00,1,0.00,0
22497,SUCURSAL NORTE,DAE04407025,C,C,True,0.0,0.000000,0.00,0,0.00,0
18305,SUCURSAL MEDELLIN,DAE06979136,C,C,True,0.0,0.000000,0.00,0,0.00,0
14593,SUCURSAL MEDELLIN,BCE00710125,C,C,True,0.0,0.000000,0.00,0,0.00,0
23805,SUCURSAL VALLADOLID,BAE01141125,C,C,True,0.0,0.000000,0.00,0,0.00,0
1024,SUCURSAL BARRANQUILLA,BHE00742125,C,C,True,0.0,0.000000,0.12,1,0.09,1
8701,SUCURSAL CALI,DAC00037189,C,B,False,0.0,2.187032,6.03,9,4.69,7
20840,SUCURSAL NORTE,BCS00030125,C,B,False,0.0,0.532734,1.21,2,0.97,2
